# Gestor de investimento
---

In [53]:
import pandas as pd
import numpy as np
import os
import xlrd

import plotly.graph_objs as go

# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [46]:
# Função para pegar os dados de Papel (ações)
def get_stocks(df):
    stocks = []

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Papel' or row['Unnamed: 2'] == 'Ações':
            continue
        elif row['Unnamed: 2'] == 'Opções':
            break
        else:
            stocks.append(row[['Unnamed: 2', 'Unnamed: 10', 'Unnamed: 18', 
                                'Unnamed: 28', 'Unnamed: 35', 'Unnamed: 43', 
                                'Unnamed: 51', 'Unnamed: 61', 'Unnamed: 68', 
                                'Unnamed: 76', 'Unnamed: 83']].values)

    pd_stock = pd.DataFrame(columns=['Papel', 'Qtd Disponivel', 
                                    'Qtd Projetado', 'Qtd Dia', 
                                    'Qtd Garantia BOV', 'Qtd Garantia BMF', 
                                    'Qtd Estruturados', 'Liq Termo', 
                                    'Qtd Total', 'Cotacao', 'Financeiro'], data=stocks)

    return pd_stock


def get_picking(df):
    picking = []
    start = False

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Proventos de Ação':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Papel':
            continue
        elif row['Unnamed: 2'] == 'Renda Fixa':
            break
        elif start == True:
            picking.append(row[['Unnamed: 2', 'Unnamed: 11', 
                            'Unnamed: 22', 'Unnamed: 60', 
                            'Unnamed: 77']].values)

    pd_picking = pd.DataFrame(columns=['Papel', 'Qtd Provisionada', 
                                    'Tipo', 'Data Pagamento', 'Valor'], data=picking)
  
    return pd_picking


def get_fi(df):
    x = []
    start = False

    for index, row in df[~df.isnull()].iterrows():
        if row['Unnamed: 2'] == 'Fundos de Investimentos':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Nome Fundo':
            continue
        elif row['Unnamed: 2'] == 'Posição de Fundos Imobiliários':
            break
        elif (start == True) and ((type(row['Unnamed: 2']) == str)):
            if type(row['Unnamed: 2']) == str: 
                fi_name = np.array([row['Unnamed: 2']])
                data = df[df.index == index +1][['Unnamed: 13', 'Unnamed: 19', 
                                                    'Unnamed: 33', 'Unnamed: 40', 
                                                    'Unnamed: 48', 'Unnamed: 54', 
                                                    'Unnamed: 60', 'Unnamed: 70', 
                                                    'Unnamed: 81']]
                
                data.insert(0, 'Nome FI', row['Unnamed: 2'])
                # print(data.values[0])
                x.append(data.values[0])
                #display(data.head())
    
    pd_fi = pd.DataFrame(columns=['Nome', 'Data', 'Qtd Cotas', 
                                    'Valor Cota', 'Valor Bruto', 
                                    'IR', 'IOF', 'Valor Liquido', 
                                    'Aplicacao Pendente', 'Total Bruto'], data=x)
    return pd_fi


def get_fii(df):
    fii = []
    start = False

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Posição de Fundos Imobiliários':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Nome':
            continue
        elif row['Unnamed: 2'] == 'Proventos de Fundo Imobiliário':
            break
        elif start == True:
            fii.append(row[['Unnamed: 2', 'Unnamed: 14', 
                            'Unnamed: 26', 'Unnamed: 38', 
                            'Unnamed: 45', 'Unnamed: 55', 
                            'Unnamed: 74']].values)

    df_result = pd.DataFrame(columns=['Papel', 'Qtd Disponivel', 
                                    'Qtd Projetada', 'Qtd Dia', 'Qtde Total', 
                                    'Ult Cotacao', 'Financeiro'], data=fii)
  
    return df_result


def get_picking_fii(df):
    p_fii = []
    start = False

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Proventos de Fundo Imobiliário':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Papel':
            continue
        elif row['Unnamed: 2'] == 'Clubes de Investimentos':
            break
        elif start == True:
            p_fii.append(row[['Unnamed: 2', 'Unnamed: 11', 
                            'Unnamed: 22', 'Unnamed: 60', 
                            'Unnamed: 77']].values)

    df_result = pd.DataFrame(columns=['Papel', 'Tipo', 
                                    'Qtd Provisionada', 'Dt Pagamento', 
                                    'Valor Provisionado'], data=p_fii)
  
    return df_result


def get_extract():
    df = pd.read_csv('../datasets/extrato/Extrato2.csv', sep=';', encoding='iso-8859-1', decimal=',')
    df['Mov'] = pd.to_datetime(df['Mov'], format='%d/%m/%Y')
    df['Liq'] = pd.to_datetime(df['Liq'], format='%d/%m/%Y')
    df.rename(columns={'Hist__o': 'Descricao'}, inplace=True)

    return df

In [3]:
files = os.listdir('../datasets/posicao/')

stocks = pd.DataFrame()
pickings = pd.DataFrame()
fis = pd.DataFrame()
fiis = pd.DataFrame()
picking_fii = pd.DataFrame()

for file_name in files:
    if file_name == '.DS_Store':
        continue
    
    wb = xlrd.open_workbook('../datasets/posicao/' + file_name, logfile=open(os.devnull, 'w'))
    df = pd.read_excel(wb)
    
    #df = pd.read_excel('../datasets/' + file_name)
    date_position = df[df['Unnamed: 56'].str.contains('Data de referência', na=False)]['Unnamed: 56']
    
    # position date
    date_position = pd.to_datetime(date_position.str.replace('Data de referência: ', ''), format='%d/%m/%Y')
    
    #month = int(date_position.dt.month.values)
    #year = int(date_position.dt.year.values)
    #period = str(year) + '/' + str(month)
    period = date_position.values[0]

    
    # get stocks
    df_stocks = get_stocks(df)
    #display(df_stocks.head())
    df_stocks['period'] = period
    stocks = stocks.append(df_stocks, ignore_index=True)

    df_pickings = get_picking(df)
    df_pickings['period'] = period
    pickings = pickings.append(df_pickings, ignore_index=True)

    df_fi = get_fi(df)
    df_fi['period'] = period
    fis = fis.append(df_fi, ignore_index=True)

    df_fii = get_fii(df)
    df_fii['period'] = period
    fiis = fiis.append(df_fii, ignore_index=True)

    df_picking_fii = get_picking_fii(df)
    df_picking_fii['period'] = period

    picking_fii = picking_fii.append(df_picking_fii, ignore_index=True)

# display(stocks.sort_values('period', ascending=False))
# display(fi)

In [4]:
picking_fii.sort_values(by='period').head()

,Papel,Tipo,Qtd Provisionada,Dt Pagamento,Valor Provisionado,period
17,RNGO11,RENDIMENTO,49,14/02/2017,30.38,2017-01-31
16,RNGO11,RENDIMENTO,49,14/03/2017,29.40,2017-02-24
23,RNGO11,RENDIMENTO,49,17/04/2017,29.40,2017-03-31
21,RNGO11,RENDIMENTO,49,15/05/2017,24.99,2017-04-28
12,RNGO11,RENDIMENTO,49,14/06/2017,26.46,2017-05-31


In [5]:
fis['period'].sort_values().unique()

array(['2016-04-29T00:00:00.000000000', '2016-05-31T00:00:00.000000000',
       '2016-06-30T00:00:00.000000000', '2016-07-29T00:00:00.000000000',
       '2016-08-31T00:00:00.000000000', '2016-09-30T00:00:00.000000000',
       '2016-10-31T00:00:00.000000000', '2016-11-30T00:00:00.000000000',
       '2016-12-29T00:00:00.000000000', '2017-01-31T00:00:00.000000000',
       '2017-02-24T00:00:00.000000000', '2017-03-31T00:00:00.000000000',
       '2017-04-28T00:00:00.000000000', '2017-05-31T00:00:00.000000000',
       '2017-06-30T00:00:00.000000000', '2017-07-31T00:00:00.000000000',
       '2017-08-31T00:00:00.000000000', '2017-09-29T00:00:00.000000000',
       '2017-10-31T00:00:00.000000000', '2017-11-30T00:00:00.000000000',
       '2017-12-28T00:00:00.000000000', '2018-01-31T00:00:00.000000000',
       '2018-02-28T00:00:00.000000000', '2018-03-29T00:00:00.000000000',
       '2018-04-30T00:00:00.000000000', '2018-05-30T00:00:00.000000000',
       '2018-06-29T00:00:00.000000000', '2018-07-31

In [6]:
display(stocks.head())
display(pickings.head())
display(fis.head())
display(fiis.head())
display(picking_fii.head())

,Papel,Qtd Disponivel,Qtd Projetado,Qtd Dia,Qtd Garantia BOV,Qtd Garantia BMF,Qtd Estruturados,Liq Termo,Qtd Total,Cotacao,Financeiro,period
0,JHSF3,1600,0,0,0,0,0,0,1600,7.01,"11,216.00",2020-02-28
1,MGLU3,800,0,0,0,0,0,0,800,50.52,"40,416.00",2020-02-28
2,MGLU3,800,0,0,0,0,0,0,800,37.04,"29,632.00",2019-09-30
3,BPAC11,200,0,0,0,0,0,0,200,42.30,"8,460.00",2020-04-30
4,JHSF3,1600,0,0,0,0,0,0,1600,4.16,"6,656.00",2020-04-30


,Papel,Qtd Provisionada,Tipo,Data Pagamento,Valor,period
0,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,28.63,2020-02-28
1,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,58.88,2020-02-28
2,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,28.63,2020-04-30
3,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,58.88,2020-04-30
4,WEGE3,300,JUROS SOBRE CAPITAL PROPRIO,12/08/2020,9.07,2020-04-30


,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period
0,XP Long Short FIC FIM,2020-02-28,"6,902.02",2.91,"20,105.88",38.48,0,"20,067.40",0,"20,105.88",2020-02-28
1,Equitas Selection FIC FIA,2020-02-28,"3,018.52",7.23,"21,812.68",286.90,0,"21,525.78",0,"21,812.68",2020-02-28
2,AZ Quest Multi FIC FIM,2020-02-28,"1,464.80",12.95,"18,967.40",93.33,0,"18,874.07",0,"18,967.40",2020-02-28
3,Mauá Macro FIC FIM,2020-02-28,37.17,787.62,"29,278.24",186.19,0,"29,092.05",0,"29,278.24",2020-02-28
4,Bahia AM Maraú FIC de FIM,2020-02-28,"33,810.43",2.56,"86,466.95",67.77,0,"86,399.18",0,"86,466.95",2020-02-28


,Papel,Qtd Disponivel,Qtd Projetada,Qtd Dia,Qtde Total,Ult Cotacao,Financeiro,period
0,IBFF11,50,0,0,50.00,100.10,"5,005.00",2020-02-28
1,BCFF11,8,0,0,8.00,98.17,785.36,2020-02-28
2,CNES11,11,0,0,11.00,83.50,918.50,2020-02-28
3,OULG11B,50,0,0,50.00,105.00,"5,250.00",2020-02-28
4,RNGO11,49,0,0,49.00,96.61,"4,733.89",2020-02-28


,Papel,Tipo,Qtd Provisionada,Dt Pagamento,Valor Provisionado,period
0,RNGO11,RENDIMENTO,49,13/03/2020,23.03,2020-02-28
1,RNGO11,RENDIMENTO,49,14/10/2019,24.01,2019-09-30
2,RNGO11,RENDIMENTO,49,14/08/2019,24.01,2019-07-31
3,RNGO11,RENDIMENTO,49,15/07/2019,26.95,2019-06-28
4,RNGO11,RENDIMENTO,49,15/05/2020,20.58,2020-04-30


# Analise de fundo imobiliario

In [7]:
fis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Nome                232 non-null    object        
 1   Data                232 non-null    datetime64[ns]
 2   Qtd Cotas           232 non-null    float64       
 3   Valor Cota          232 non-null    float64       
 4   Valor Bruto         232 non-null    float64       
 5   IR                  232 non-null    float64       
 6   IOF                 232 non-null    object        
 7   Valor Liquido       232 non-null    float64       
 8   Aplicacao Pendente  232 non-null    object        
 9   Total Bruto         232 non-null    float64       
 10  period              232 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(6), object(3)
memory usage: 20.1+ KB


In [8]:
fis.sort_values(['Nome', 'period']).head()

,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period
133,AZ Quest Multi FIC FIM,2017-03-31,"1,516.42",9.95,"15,094.10",9.94,0,"15,034.29",0,"15,094.10",2017-03-31
122,AZ Quest Multi FIC FIM,2017-04-28,"1,516.42",10.00,"15,160.90",36.20,0,"15,124.70",0,"15,160.90",2017-04-28
82,AZ Quest Multi FIC FIM,2017-05-31,"1,516.42",9.73,"14,749.34",0.00,0,"14,749.34",0,"14,749.34",2017-05-31
86,AZ Quest Multi FIC FIM,2017-06-30,"1,516.42",9.80,"14,853.49",0.00,0,"14,853.49",0,"14,853.49",2017-06-30
108,AZ Quest Multi FIC FIM,2017-07-31,"1,516.42",10.15,"15,389.53",87.64,0,"15,301.89",0,"15,389.53",2017-07-31


In [17]:
fis[fis['Nome'] == 'AZ Quest Multi FIC FIM'].sort_values('period')

,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period
133,AZ Quest Multi FIC FIM,2017-03-31,"1,516.42",9.95,"15,094.10",9.94,0,"15,034.29",0,"15,094.10",2017-03-31
122,AZ Quest Multi FIC FIM,2017-04-28,"1,516.42",10.00,"15,160.90",36.20,0,"15,124.70",0,"15,160.90",2017-04-28
82,AZ Quest Multi FIC FIM,2017-05-31,"1,516.42",9.73,"14,749.34",0.00,0,"14,749.34",0,"14,749.34",2017-05-31
86,AZ Quest Multi FIC FIM,2017-06-30,"1,516.42",9.80,"14,853.49",0.00,0,"14,853.49",0,"14,853.49",2017-06-30
108,AZ Quest Multi FIC FIM,2017-07-31,"1,516.42",10.15,"15,389.53",87.64,0,"15,301.89",0,"15,389.53",2017-07-31
98,AZ Quest Multi FIC FIM,2017-08-31,"1,516.42",10.32,"15,644.93",145.10,0,"15,499.83",0,"15,644.93",2017-08-31
67,AZ Quest Multi FIC FIM,2017-09-29,"1,516.42",10.55,"16,004.66",200.93,0,"15,803.73",0,"16,004.66",2017-09-29
51,AZ Quest Multi FIC FIM,2017-10-31,"1,516.42",10.51,"15,934.88",186.97,0,"15,747.91",0,"15,934.88",2017-10-31
35,AZ Quest Multi FIC FIM,2017-11-30,"1,503.23",10.50,"15,784.72",46.16,0,"15,738.56",0,"15,784.72",2017-11-30
44,AZ Quest Multi FIC FIM,2017-12-29,"1,503.23",10.62,"15,969.51",83.11,0,"15,886.40",0,"15,969.51",2017-12-28


In [39]:
# Estou aqui!! Criar grafico comparando fundos por valor bruto

def plot_fis(df):

    for fi in df['Nome'].unique():
        fig = go.Figure()
        df_ = df[df['Nome'] == fi].sort_values('period')

        fig.add_trace(go.Scatter(x=df_['period'], y=df_['Qtd Cotas'] * df_['Valor Cota'], 
                                connectgaps=True,
                                mode='lines+markers',
                                name=fi, 
                                #text=round(df[mkt].astype(float)/1000000, 2).astype(str) + 'M', 
                                textposition="top center",
                                textfont={'color': 'rgb(100,100,100)', 'size':9}, 
                                # fill='tonexty', 
                                # df_offplatformByWeek['2020 10'].map('{:,.0f}'.format), 
                                marker=dict(size=3), 
                                opacity=.8))

        fig.update_layout(legend_orientation="h",
                        title=fi, 
                        height=300, 
                        width=600, 
                        legend=dict(font=dict(size=9)), 
                        template='plotly_white'                
        )

        fig.show()

plot_fis(fis)

# Analise aportes

In [48]:
extract = get_extract()

In [61]:
fis.Nome.unique()

array(['XP Long Short FIC FIM', 'Equitas Selection FIC FIA',
       'AZ Quest Multi FIC FIM', 'Mauá Macro FIC FIM',
       'Bahia AM Maraú FIC de FIM', 'XP Macro FIM',
       'Polo Norte I FIC FIM', 'Mirae Asset Multimercado '], dtype=object)

In [85]:
fi_map = {
    'Equitas': 'Equitas Selection FIC FIA', 
    'Polo Norte': 'Polo Norte I FIC FIM', 
    'XP Macro': 'XP Macro FIM', 
    'Bahia AM Mara': 'Bahia AM Maraú FIC de FIM', 
    'QuestMult': 'AZ Quest Multi FIC FIM', 
    'Mau Macro FIC FIM': 'Mauá Macro FIC FIM', 
    'MiraeMacroStrategy': 'Mirae Asset Multimercado ', 
    'XP LONG SHORT': 'XP Long Short FIC FIM', 
    'XP GlobCredit': 'XP GlobCredit FICFIM', # Falta posicao deste periodo
    'Vot.FicFi CambialD': 'Vot.FicFi CambialD',  # idem
    'BNP Inflacao Firf': 'BNP Inflacao Firf' # idem
}


def like_function(x):
    group = "unknown"
    for key in fi_map:
        if key in x:
            group = fi_map[key]
            break
    return group


def get_fi_extract(df):
    df_  = df[df['Descricao'].str.contains('TED APLICA')]
    df_['Nome'] = df_['Descricao'].apply(like_function)

    df_draw = df[df['Descricao'].str.contains('RESGATE')]
    df_draw['Nome'] = df_draw['Descricao'].apply(like_function)
    #df_['Nome'] = df_['Descricao'].map(lambda x: x.str.contains(fi_dict[0]))
    return df_, df_draw

In [86]:
df_fi_stock, df_fi_draw = get_fi_extract(extract)

print('-- APORTE -- ')
display(df_fi_stock.head())
display(df_fi_stock.groupby('Nome').agg({'Valor': 'sum'}).reset_index())
display(df_fi_stock['Valor'].sum())


print('-- RESGATE -- ')
display(df_fi_draw.head())
display(df_fi_draw.groupby('Nome').agg({'Valor': 'sum'}).reset_index())
display(df_fi_draw['Valor'].sum())


-- APORTE -- 


,Liq,Mov,Descricao,Valor,Saldo,Nome
60,2019-09-12,2019-09-12,TED TER BCO 017 AGE 0001 CTA 1540 7 - TED APLICAÌO FUNDOS Equitas Selection FIC,"-19,900.00","27,647.52",Equitas Selection FIC FIA
86,2018-12-27,2018-12-27,TED TER BCO 017 AGE 0001 CTA 1310 2 - TED APLICAÌO FUNDOS Polo Norte I FIC FIM,"-30,000.00",305.63,Polo Norte I FIC FIM
108,2018-06-15,2018-06-15,TED TER BCO 017 AGE 0001 CTA 1603 9 - TED APLICAÌO FUNDOS XP Macro FIM,"-10,000.00","9,123.03",XP Macro FIM
117,2018-04-30,2018-04-30,TED TER BCO 017 AGE 0001 CTA 1603 9 - TED APLICAÌO FUNDOS XP Macro FIM,"-19,500.00","-19,004.10",XP Macro FIM
139,2017-10-10,2017-10-10,TED TER BCO 237 AGE 2856 CTA 4563 2 - TED APLICAÌO FUNDOS Bahia AM Mara FIC FI,"-23,000.00",259.50,Bahia AM Maraú FIC de FIM


,Nome,Valor
0,AZ Quest Multi FIC FIM,"-15,000.00"
1,BNP Inflacao Firf,"-10,300.00"
2,Bahia AM Maraú FIC de FIM,"-73,000.00"
3,Equitas Selection FIC FIA,"-19,900.00"
4,Mauá Macro FIC FIM,"-23,500.00"
5,Mirae Asset Multimercado,"-16,000.00"
6,Polo Norte I FIC FIM,"-30,000.00"
7,Vot.FicFi CambialD,"-3,000.00"
8,XP GlobCredit FICFIM,"-33,000.00"
9,XP Long Short FIC FIM,"-15,000.00"


-268200.0

-- RESGATE -- 


,Liq,Mov,Descricao,Valor,Saldo,Nome
20,2020-03-16,2020-03-16,IRRF S/RESGATE FUNDOS QuestMult FI CotasFI,-37.00,"24,064.50",AZ Quest Multi FIC FIM
21,2020-03-16,2020-03-16,RESGATE QuestMult FI CotasFI,"18,591.84","24,101.50",AZ Quest Multi FIC FIM
111,2018-06-11,2018-06-11,RESGATE XP Macro FIM,"18,995.54","19,092.49",XP Macro FIM
115,2018-04-30,2018-04-30,IRRF S/RESGATE FUNDOS MiraeMacroStrategy,-54.81,63.80,Mirae Asset Multimercado
116,2018-04-30,2018-04-30,RESGATE MiraeMacroStrategy,"19,122.71",118.61,Mirae Asset Multimercado


,Nome,Valor
0,AZ Quest Multi FIC FIM,"18,554.84"
1,BNP Inflacao Firf,"10,071.94"
2,Mauá Macro FIC FIM,-111.82
3,Mirae Asset Multimercado,"18,633.40"
4,Vot.FicFi CambialD,"2,952.78"
5,XP GlobCredit FICFIM,"31,029.08"
6,XP Long Short FIC FIM,-406.29
7,XP Macro FIM,"18,995.54"
8,unknown,"24,965.09"


124684.56000000003

In [45]:
extract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Liq        340 non-null    datetime64[ns]
 1   Mov        340 non-null    datetime64[ns]
 2   Descrição  340 non-null    object        
 3   Valor      340 non-null    float64       
 4   Saldo      340 non-null    float64       
dtypes: datetime64[ns](2), float64(2), object(1)
memory usage: 13.4+ KB


In [77]:
extract[extract['Descricao'].str.contains('QuestMult')].sort_values('Mov')




,Liq,Mov,Descricao,Valor,Saldo
193,2017-03-17,2017-03-17,TED TER BCO 017 AGE 0001 CTA 1518 0 - TED APLICAÌO FUNDOS QuestMult FI CotasFI,"-15,000.00",80.67
20,2020-03-16,2020-03-16,IRRF S/RESGATE FUNDOS QuestMult FI CotasFI,-37.00,"24,064.50"
21,2020-03-16,2020-03-16,RESGATE QuestMult FI CotasFI,"18,591.84","24,101.50"


In [57]:
display(extract.groupby('Descricao').agg({'Valor':'sum'}).reset_index().sort_values('Valor', ascending=False).head())
display(extract.groupby('Descricao').agg({'Valor':'sum'}).reset_index().sort_values('Valor', ascending=False).tail(5)) # Aportes para fundos

,Descricao,Valor
74,TED - RECEBIMENTO DE TED - SPB,"228,400.00"
52,RESGATE XP GlobCredit FICFIM,"31,029.08"
47,RESGATE MiraeMacroStrategy,"19,122.71"
53,RESGATE XP Macro FIM,"18,995.54"
49,RESGATE QuestMult FI CotasFI,"18,591.84"


,Descricao,Valor
24,OPERAÍES EM BOLSA PR 12/09/2019 NOTA N¼ 16418593,"-27,630.26"
81,TED TER BCO 017 AGE 0001 CTA 1603 9 - TED APLICAÌO FUNDOS XP Macro FIM,"-29,500.00"
78,TED TER BCO 017 AGE 0001 CTA 1310 2 - TED APLICAÌO FUNDOS Polo Norte I FIC FIM,"-30,000.00"
84,TED TER BCO 208 AGE 0001 CTA 183448 - TED APLICAÌO FUNDOS XP GlobCredit FICFIM,"-33,000.00"
85,TED TER BCO 237 AGE 2856 CTA 4563 2 - TED APLICAÌO FUNDOS Bahia AM Mara FIC FI,"-73,000.00"


In [55]:
display(extract.groupby(['Descricao', 'Mov']).agg({'Valor':'sum'}).reset_index().sort_values('Valor', ascending=False).tail(30))

,Descricao,Mov,Valor
30,IRRF S/ RESGATE FUNDOS - COME COTAS Mau Macro FIC FIM,2017-06-01,-109.38
34,IRRF S/ RESGATE FUNDOS - COME COTAS XP LONG SHORT,2016-06-01,-132.97
32,IRRF S/ RESGATE FUNDOS - COME COTAS MiraeMacroStrategy,2016-12-05,-180.48
43,OPERAÍES EM BOLSA LIQ. D+1 PR 24/05/2017 NOTA N¼ 8920878,2017-05-24,-193.66
33,IRRF S/ RESGATE FUNDOS - COME COTAS MiraeMacroStrategy,2017-06-01,-197.23
35,IRRF S/ RESGATE FUNDOS - COME COTAS XP LONG SHORT,2016-12-05,-242.19
27,IRRF S/ RESGATE FUNDOS - COME COTAS Azul QuantitativoFIM,2015-06-01,-251.79
18,DEPOSITO DE MARGEM DE GARANTIA,2017-06-19,-578.89
337,TED TER BCO 655 AGE 0001 CTA 1296001 5 - TED APLICAÌO FUNDOS Vot.FicFi CambialD,2015-09-10,"-3,000.00"
323,TED BCO 341 AGE 0407 CTA 03400 8 - RETIRADA EM C/C,2015-12-03,"-3,000.00"
